In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass
import logging
from typing import Dict, Optional, Protocol, Tuple
from scipy import stats
from new_strategy import TradingStrategy, Asset, BetSizingMethod, get_bet_sizing
import new_strategy
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
@dataclass
class ReturnMetrics:
    """Performance metrics for strategy evaluation"""
    total_pnl: float
    total_return_pct: float
    win_rate: float
    avg_win: float
    avg_loss: float
    sharpe: Optional[float]
    skewness: Optional[float]
    excess_kurtosis: Optional[float]
    max_drawdown_pct: float
    total_trades: int
    risk_amount: float

class Backtest:
    def __init__(self, strategy, output_dir: Path = None):
        self.strategy = strategy
        self.trades_df = strategy.get_trade_data()
        self.output_dir = output_dir or Path("data/results")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.results = {}

        self.logger = logging.getLogger(__name__)
        if not self.logger.handlers:
            logging.basicConfig(
                level=logging.INFO,
                format='%(asctime)s - %(levelname)s - %(message)s'
            )


    def _calculate_performance_metrics(self, sharpe_returns: pd.Series, full_returns: pd.Series) -> Tuple[float, float, float]:
        """
        Calculate Sharpe (from trading days) and skewness/kurtosis (from full daily returns).

        Args:
            sharpe_returns (pd.Series): Returns on actual trading days only (no forward fill).
            full_returns (pd.Series): Full daily returns (with forward fill).

        Returns:
            (sharpe, skewness, excess_kurtosis)
        """
        sharpe_returns = sharpe_returns.dropna()
        full_returns = full_returns.dropna()

        # Sharpe: only on trading days
        if len(sharpe_returns) >= 2:
            mean_return = sharpe_returns.mean()
            vol = sharpe_returns.std()
            sharpe = (mean_return / vol) * np.sqrt(252) if vol != 0 else np.nan
        else:
            sharpe = np.nan

        # Skewness & kurtosis: on trading days
        if len(full_returns) >= 2:
            skewness = stats.skew(sharpe_returns, nan_policy='omit')
            excess_kurtosis = stats.kurtosis(sharpe_returns, fisher=True, nan_policy='omit')
        else:
            skewness = np.nan
            excess_kurtosis = np.nan

        return sharpe, skewness, excess_kurtosis


    def _calculate_drawdown(self, returns: pd.Series) -> float:
        cum_returns = (1 + returns).cumprod()
        rolling_max = cum_returns.expanding().max()
        drawdowns = (cum_returns - rolling_max) / rolling_max
        return abs(drawdowns.min()) * 100

    def _calculate_return_metrics(self, trades_df: pd.DataFrame) -> Tuple[ReturnMetrics, pd.DataFrame]:
        """
        Calculate metrics based on capital growth curve and update capital_curve column.
        """
        if trades_df.empty:
            return ReturnMetrics(0, 0, 0, 0, 0, np.nan, np.nan, np.nan, 0, 0, 0), trades_df

        #Filter Trades that are 0
        trades_df = trades_df[trades_df['position_size'] > 0]

        # Sort trades by exit time to maintain order
        trades_df = trades_df.sort_values('exit_time')
        capital = self.strategy.INITIAL_CAPITAL
        capital_curve = []

        # Build capital curve on trade exits
        for _, trade in trades_df.iterrows():
            capital += trade['pnl']
            capital_curve.append(capital)

        trades_df = trades_df.copy()
        trades_df['capital_curve'] = capital_curve

        # Capital curve: trading days only (no forward fill)
        capital_curve_series = trades_df.set_index('exit_time')['capital_curve'].sort_index()
        returns_on_trading_days = capital_curve_series.pct_change().dropna()

        # Full daily capital curve (with forward fill for skew/kurtosis)
        capital_df = trades_df[['exit_time', 'capital_curve']].copy()
        capital_df['date'] = capital_df['exit_time'].dt.date
        daily_capital = capital_df.groupby('date')['capital_curve'].last()

        full_date_range = pd.date_range(daily_capital.index.min(), daily_capital.index.max(), freq='D')
        daily_capital_filled = daily_capital.reindex(full_date_range).ffill()
        returns_full = daily_capital_filled.pct_change().dropna()

        # Calculate performance metrics
        sharpe, skewness, excess_kurtosis = self._calculate_performance_metrics(
            sharpe_returns=returns_on_trading_days,
            full_returns=returns_full
        )

        # Basic metrics
        winning_trades = trades_df[trades_df['pnl'] > 0]
        losing_trades = trades_df[trades_df['pnl'] < 0]
        total_pnl = trades_df['pnl'].sum()
        initial_capital = self.strategy.INITIAL_CAPITAL

        return ReturnMetrics(
            total_pnl=total_pnl,
            total_return_pct=((capital - initial_capital) / initial_capital) * 100,
            win_rate=len(winning_trades) / len(trades_df),
            avg_win=winning_trades['pnl'].mean() if not winning_trades.empty else 0,
            avg_loss=losing_trades['pnl'].mean() if not losing_trades.empty else 0,
            sharpe=sharpe,
            skewness=skewness,
            excess_kurtosis=excess_kurtosis,
            max_drawdown_pct=self._calculate_drawdown(returns_full),
            total_trades=len(trades_df),
            risk_amount=trades_df['risk_amount'].iloc[0] if 'risk_amount' in trades_df.columns else 0
        ), trades_df


    def _export_detailed_trades(self) -> None:
        bet_sizing_name = self.strategy.bet_sizing_method.value
        filename = f"trades_detailed_{self.strategy.asset.value}_{bet_sizing_name}.csv"
        output_path = self.output_dir / filename
        self.trades_df.sort_values('entry_time').to_csv(output_path, index=False)
        self.logger.info(f"Exported detailed trades to {output_path}")


    def save_results_to_file(self) -> None:
        """Save backtest results to a text file (no fee, includes bet sizing method)"""
        bet_sizing_name = self.strategy.bet_sizing_method.value
        filename = f"backtest_results_{self.strategy.asset.value}_{bet_sizing_name}.txt"
        output_path = self.output_dir / filename

        with open(output_path, 'w') as f:
            f.write(f"=== {self.strategy.asset.value} Backtest Results ===\n")
            f.write(f"Bet Sizing Strategy: {bet_sizing_name}\n\n")

            if self.results['period']['start'] is not None:
                f.write(f"Period: {self.results['period']['start']:%Y-%m-%d} to {self.results['period']['end']:%Y-%m-%d}\n\n")

            for session, data in self.results['sessions'].items():
                metrics = data['metrics']
                f.write(f"\n{session.upper()} Session Performance:\n")
                f.write(f"Initial Capital: ${self.strategy.INITIAL_CAPITAL:,.2f}\n")
                f.write(f"Final Capital: ${(self.strategy.INITIAL_CAPITAL * (1 + metrics.total_return_pct/100)):,.2f}\n")
                f.write(f"Total PnL: ${metrics.total_pnl:,.2f}\n")
                f.write(f"Return: {metrics.total_return_pct:.2f}%\n")

                session_trades = self.trades_df[self.trades_df['session'] == session]
                wins = len(session_trades[session_trades['pnl'] > 0])
                losses = len(session_trades[session_trades['pnl'] <= 0])
                f.write(f"Win Rate: {metrics.win_rate:.2%} ({wins}W/{losses}L)\n")
                f.write(f"Max Drawdown: {metrics.max_drawdown_pct:.2f}%\n")

                if metrics.sharpe is not None:
                    f.write(f"Sharpe Ratio: {metrics.sharpe:.2f}\n")
                if metrics.skewness is not None:
                    f.write(f"Skewness: {metrics.skewness:.3f}\n")
                if metrics.excess_kurtosis is not None:
                    f.write(f"Excess Kurtosis: {metrics.excess_kurtosis:.3f}\n")

                # Write attempt analysis
                if data['attempts']:
                    f.write("\nAttempt Analysis:\n")
                    for attempt, attempt_metrics in data['attempts'].items():
                        attempt_trades = session_trades[session_trades['attempt'] == attempt]
                        wins = len(attempt_trades[attempt_trades['pnl'] > 0])
                        losses = len(attempt_trades[attempt_trades['pnl'] <= 0])

                        f.write(f"\n  Attempt {attempt}:\n")
                        f.write(f"  Trades: {attempt_metrics.total_trades} "
                                f"({attempt_metrics.total_trades/metrics.total_trades*100:.1f}% of session trades)\n")
                        f.write(f"  PnL: ${attempt_metrics.total_pnl:,.2f}\n")
                        f.write(f"  Win Rate: {attempt_metrics.win_rate:.2%} ({wins}W/{losses}L)\n")
                        f.write(f"  Average Win: ${attempt_metrics.avg_win:,.2f}\n")
                        f.write(f"  Average Loss: ${attempt_metrics.avg_loss:,.2f}\n")

            f.write("\n" + "="*50 + "\n")

        self.logger.info(f"Saved backtest results to {output_path}")


    def run_analysis(self) -> None:
        if self.trades_df is None or self.trades_df.empty:
            self.logger.warning(f"No trades generated for {self.strategy.asset.value}")
            self.results = {
                'asset': self.strategy.asset.value,
                'sessions': {},
                'period': {'start': None, 'end': None}
            }
            return

        # Initialize results structure
        self.results = {
            'asset': self.strategy.asset.value,
            'sessions': {},
            'period': {
                'start': self.trades_df['entry_time'].min(),
                'end': self.trades_df['entry_time'].max()
            }
        }

        for session in ['asian', 'london', 'us']:
            session_trades = self.trades_df[self.trades_df['session'] == session]

            if session_trades.empty:
                self.logger.debug(f"No trades for {session} session in {self.strategy.asset.value}")
                continue

            session_metrics, updated_trades = self._calculate_return_metrics(session_trades)

            # Update main trades_df with the capital curve for that session
            self.trades_df.loc[updated_trades.index, 'capital_curve'] = updated_trades['capital_curve']

            attempt_metrics = {}
            for attempt in sorted(session_trades['attempt'].unique()):
                attempt_trades = session_trades[session_trades['attempt'] == attempt]
                metrics, _ = self._calculate_return_metrics(attempt_trades)
                attempt_metrics[attempt] = metrics

            self.results['sessions'][session] = {
                'metrics': session_metrics,
                'attempts': attempt_metrics
            }

        # Export after all sessions processed
        self._export_detailed_trades()
        self.save_results_to_file()

    def print_summary(self) -> None:
        """Print comprehensive analysis summary (no fee, includes bet sizing)"""
        if not self.results:
            self.logger.warning("No results available for analysis")
            return

        bet_sizing_name = type(self.strategy.bet_sizing).__name__.lower()

        print(f"\n=== {self.strategy.asset.value} Backtest Results ===")
        print(f"Bet Sizing Strategy: {bet_sizing_name}\n")

        if not self.results['sessions']:
            print("No trades were generated during the test period.")
            return

        if self.results['period']['start'] is not None:
            print(f"Period: {self.results['period']['start']:%Y-%m-%d} to {self.results['period']['end']:%Y-%m-%d}")

        for session, data in self.results['sessions'].items():
            metrics = data['metrics']
            print(f"\n{session.upper()} Session Performance:")
            print(f"Initial Capital: ${self.strategy.INITIAL_CAPITAL:,.2f}")
            print(f"Final Capital: ${(self.strategy.INITIAL_CAPITAL * (1 + metrics.total_return_pct / 100)):,.2f}")
            print(f"Total PnL: ${metrics.total_pnl:,.2f}")
            print(f"Return: {metrics.total_return_pct:.2f}%")

            session_trades = self.trades_df[self.trades_df['session'] == session]
            wins = len(session_trades[session_trades['pnl'] > 0])
            losses = len(session_trades[session_trades['pnl'] <= 0])
            print(f"Win Rate: {metrics.win_rate:.2%} ({wins}W/{losses}L)")
            print(f"Max Drawdown: {metrics.max_drawdown_pct:.2f}%")

            if metrics.sharpe is not None:
                print(f"Sharpe Ratio: {metrics.sharpe:.2f}")
            if metrics.skewness is not None:
                print(f"Skewness: {metrics.skewness:.3f}")
            if metrics.excess_kurtosis is not None:
                print(f"Excess Kurtosis: {metrics.excess_kurtosis:.3f}")

            if data['attempts']:
                print("\nAttempt Analysis:")
                for attempt, attempt_metrics in data['attempts'].items():
                    attempt_trades = session_trades[session_trades['attempt'] == attempt]
                    wins = len(attempt_trades[attempt_trades['pnl'] > 0])
                    losses = len(attempt_trades[attempt_trades['pnl'] <= 0])

                    print(f"\n  Attempt {attempt}:")
                    print(f"  Trades: {attempt_metrics.total_trades} "
                        f"({attempt_metrics.total_trades / metrics.total_trades * 100:.1f}% of session trades)")
                    print(f"  PnL: ${attempt_metrics.total_pnl:,.2f}")
                    print(f"  Win Rate: {attempt_metrics.win_rate:.2%} ({wins}W/{losses}L)")
                    print(f"  Average Win: ${attempt_metrics.avg_win:,.2f}")
                    print(f"  Average Loss: ${attempt_metrics.avg_loss:,.2f}")
      

BET_SIZING_MODE = BetSizingMethod.FIXED

def main():
    """
    Run backtest with the updated trading strategy and zero trading fees.
    """
    from new_strategy import TradingStrategy
    from pathlib import Path
    import logging

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    logger = logging.getLogger(__name__)

    assets = ["XAUUSD"]

    output_dir = Path("data/results")
    output_dir.mkdir(parents=True, exist_ok=True)

    for asset_name in assets:
        try:
            data_path = Path(f"data/processed/{asset_name}/combined_data.csv")
            if not data_path.exists():
                logger.warning(f"Data file not found for {asset_name}, skipping...")
                continue

            data = pd.read_csv(data_path, index_col='timestamp', parse_dates=True)

            logger.info(f"\nRunning backtest for {asset_name} with no fees")

           
            past_returns = data['close'].pct_change().dropna()
            bet_sizing = get_bet_sizing(BET_SIZING_MODE, past_returns=past_returns)

            strategy = TradingStrategy(data, asset_name, bet_sizing, BET_SIZING_MODE)
            strategy.generate_signals()
            strategy.simulate_trades()

            backtest = Backtest(strategy, output_dir)
            backtest.run_analysis()
            backtest.print_summary()
            #If KellyBetSizing is used, print how many times the cap was hit
            if hasattr(strategy.bet_sizing, "report_limit_hits"):
                 strategy.bet_sizing.report_limit_hits()
            if hasattr(strategy.bet_sizing, "limit_hit_counter"):
                print(f"[INFO] PercentVolatilityBetSizing limit hit {strategy.bet_sizing.limit_hit_counter} times.")

        except Exception as e:
            logger.error(f"Error testing {asset_name}: {str(e)}", exc_info=True)

if __name__ == "__main__":
    main()

2025-05-08 16:08:26,244 - INFO - 
Running backtest for XAUUSD with no fees
2025-05-08 16:08:26,300 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing
2025-05-08 16:14:14,732 - INFO - Exported detailed trades to data/results/trades_detailed_XAUUSD_fixed.csv
2025-05-08 16:14:14,802 - INFO - Saved backtest results to data/results/backtest_results_XAUUSD_fixed.txt



=== XAUUSD Backtest Results ===
Bet Sizing Strategy: fixedfractionalbetsizing

Period: 2020-01-02 to 2024-11-22

ASIAN Session Performance:
Initial Capital: $100,000.00
Final Capital: $99,975.36
Total PnL: $-24.64
Return: -0.02%
Win Rate: 49.51% (759W/774L)
Max Drawdown: 2.17%
Sharpe Ratio: 0.00
Skewness: 0.076
Excess Kurtosis: -0.881

Attempt Analysis:

  Attempt 1:
  Trades: 1277 (83.3% of session trades)
  PnL: $764.12
  Win Rate: 49.10% (627W/650L)
  Average Win: $58.90
  Average Loss: $-56.69

  Attempt 2:
  Trades: 217 (14.2% of session trades)
  PnL: $-22.80
  Win Rate: 53.00% (115W/102L)
  Average Win: $48.33
  Average Loss: $-54.72

  Attempt 3:
  Trades: 39 (2.5% of session trades)
  PnL: $-765.96
  Win Rate: 43.59% (17W/22L)
  Average Win: $39.12
  Average Loss: $-65.04

LONDON Session Performance:
Initial Capital: $100,000.00
Final Capital: $107,031.90
Total PnL: $7,031.90
Return: 7.03%
Win Rate: 49.46% (920W/940L)
Max Drawdown: 2.91%
Sharpe Ratio: 0.64
Skewness: 0.175
Exc